## Offline simulator for CV furnace

This script given the temperature and expected O2 output predicts the burner settings to be used. This works as a offline simulator to and allows hte user to interact and observe the change in burner settings for different senarios.

Import required packages:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import panel as pn
import holoviews as hv
import hvplot.pandas
import plotly.graph_objects as go
import data_processing_methods as dpm
from sklearn.gaussian_process import GaussianProcessRegressor

Fit the model (prediction model - Gaussian Process Regression)

In [2]:
# Read the dataframe
burner_settings_df = pd.read_excel('Burners_settings_vs_O2 _rev1.xlsx', sheet_name='Sheet1')
standardised_df = burner_settings_df.copy()
for col in burner_settings_df.columns:

    if (np.std(burner_settings_df[col])> 0):
        standardised_df[col] = dpm.standardise(burner_settings_df[col],np.mean(burner_settings_df[col]),np.std(burner_settings_df[col]))
    else:
        standardised_df = standardised_df.drop(columns=col)

X1 = standardised_df.drop(columns =['Output O2 / %', 'Output Burner usage / %'])
y1 = standardised_df['Output O2 / %']

# Drop columns with corr coeffcients higher than 0.9 
X1 = X1.drop(columns =['Burner turns from zero', 'Fan speed / RPM'])
X1_train = X1[10:]
y1_train = y1[10:]

# Train Gaussian Process regression model
gpr_1 = GaussianProcessRegressor().fit(X1_train, y1_train)
y1_std = np.std(burner_settings_df['Output O2 / %'])
y1_mean = np.mean(burner_settings_df['Output O2 / %'])

**Scatter plots**

Scatter plots results and visualising input parameter limits. User can define the temperature limits and O2 output limits so that application can suggest the best
parameters to be used to obtain the expected O2 level.

In [30]:
# No. samples to be generated
N_samples = 5000
cols = list(X1_train.columns)

# Initialise arrays to store samples of un-standardised and 
# standardised inputs
D = len (cols)
X_samples_us = np.zeros([N_samples, D])
X_samples = np.zeros([N_samples, D])

for i in range(N_samples):
    for j in range (D):
        X_samples_us[i, j] = np.random.uniform(np.min(burner_settings_df[cols[j]]), np.max(burner_settings_df[cols[j]]))
        
# Standardise the samples created
for j in range (D):    
    X_samples[:,j] = dpm.standardise(X_samples_us[:,j],np.mean(burner_settings_df[cols[j]]),np.std(burner_settings_df[cols[j]]))

# Save the predictions 
X_samples_df = pd.DataFrame(data = X_samples, columns = cols)
X_samples_us_df = pd.DataFrame(data = X_samples_us, columns = cols)
y_samples_predict = gpr_1.predict(X_samples_df)
y_samples_predict = y_samples_predict*y1_std+y1_mean
X_samples_us_df['Predicted O2 %'] = y_samples_predict

cols_samples = list(X_samples_us_df.columns)
y_samples = pn.widgets.Select(name='Chose a signal', options=cols_samples ,value='Temperature setpoint / degC')

# Create widgets for setting input limits
temp_min_slider = pn.widgets.IntSlider(name='Temperature min limit', start=1100, end=1150, step=1, value = 1140)
temp_max_slider = pn.widgets.IntSlider(name='Temperature max limit', start=1100, end=1150, step=1, value = 1145)
O2_min_slider = pn.widgets.IntSlider(name='O2 min limit', start=0, end=10, step=1, value = 0)
O2_max_slider = pn.widgets.IntSlider(name='O2 maxt limit', start=0, end=10, step=1, value = 2)

@pn.depends(y_samples.param.value)
def plot_main_effects(y_samples,slider_pos_temp_min,slider_pos_temp_max,slider_pos_O2_min,slider_pos_O2_max):
    # Find the mean of each bin (binning data)
    n_bins = 30
    x_plot = X_samples_us_df[y_samples]
    y_plot = X_samples_us_df['Predicted O2 %']

    bins = np.linspace(np.min(x_plot), np.max(x_plot), n_bins)
    main_effect = np.zeros(len(bins)-1)
    main_effect_index = np.zeros(len(bins)-1)
    main_effect_df = pd.DataFrame({})
    
    for j in range(len(bins)-1):
        indx = np.logical_and(x_plot > bins[j], x_plot < bins[j+1])
        main_effect_index[j] = 0.5*(bins[j] + bins[j+1])

        # Only compute mean if there are any points in bin
        if np.sum(indx) > 0:
            main_effect[j] = np.mean(y_plot[indx])

    main_effect_df['index'] = main_effect_index
    main_effect_df['value'] = main_effect
    
    if (y_samples == 'Temperature setpoint / degC'):
        vline1 = hv.VLine(int(slider_pos_temp_min))
        vline1.opts(color= '#d83569')
        vline2 = hv.VLine(int(slider_pos_temp_max))
        vline2.opts(color= '#d83569')
        
        h1line = hv.HLine(int(slider_pos_O2_min))
        h1line.opts(color='#d83569')               
        h2line = hv.HLine(int(slider_pos_O2_max))
        h2line.opts(color='#d83569')

                
    # plot main effects of model inputs
    fig_c = X_samples_us_df.hvplot.scatter(x = y_samples, y = 'Predicted O2 %', height = 500, width = 900, hover_cols = 'all')
    fig_d = main_effect_df.hvplot.line(x = 'index', y = 'value', xlim =[np.min(x_plot), np.max(x_plot)],color= 'red')
    if (y_samples == 'Temperature setpoint / degC'):
        fig = fig_c*fig_d*vline1*vline2*h1line*h2line
    
        ind1 = np.logical_and(X_samples_us_df['Temperature setpoint / degC']>slider_pos_temp_min,X_samples_us_df['Temperature setpoint / degC']<slider_pos_temp_max)
        X_samples_a_df = X_samples_us_df[ind1]
        ind2 = np.logical_and(X_samples_a_df['Predicted O2 %']<slider_pos_O2_max,X_samples_a_df['Predicted O2 %']>slider_pos_O2_min)
        sorted_df = X_samples_a_df[ind2].sort_values(by=['Predicted O2 %'],ignore_index = True)

        num1 = pn.indicators.Number(name='Fan setpoint', value=int(sorted_df['Fan setpoint / %'][0]), format='{value}%')
        num2 = pn.indicators.Number(name='Burner apporx. value', value=int(sorted_df['Burner approx value'][0]), format='{value}')
        num3 = pn.indicators.Number(name='Damper range (high)', value=int(sorted_df[' damper range / % (low)'][0]), format='{value}%')
        num4 = pn.indicators.Number(name='Damper range (low)', value=int(sorted_df['damper range / % (High)'][0]), format='{value}%')
        num = pn.Row(pn.Column(num1,num2),pn.Column(num3,num4))
        fig_out = pn.Row(fig,num)
    else:
        fig_out = fig_c*fig_d
        
    return fig_out

In [ ]:
# Plot  
explanation_pane = pn.pane.Markdown("""
# Main effects
""", width=500)

# Create interactive panels
chart_interact = pn.interact(plot_main_effects, y_samples = y_samples, slider_pos_temp_min=temp_min_slider, slider_pos_temp_max=temp_max_slider, 
                             slider_pos_O2_min=O2_min_slider, slider_pos_O2_max=O2_max_slider)
layout_b = pn.Column(explanation_pane, chart_interact)
layout_b.servable()